# Backtesting MutantSupertrend

In [1]:
# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)
import datetime
# import os.path
import sys

import pandas as pd
import numpy as np
import backtrader as bt
import backtrader.indicators as ta

import matplotlib
import tkinter
matplotlib.use('TKAgg')
# matplotlib.use('QT5Agg')

from mutant.model import MutantSupertrend
from mutant.strategy import MutantSupertrendBacktrader

raw_data_path = "../data/BTCUSD_latest.csv"
# raw_data_path = "../data/Raw_BTCUSDT1708-2303.csv"

## Load data

In [2]:
dataframe = pd.read_csv(raw_data_path,
                                parse_dates=True,
                                index_col=0)
dataframe.index = pd.to_datetime(dataframe.index, format='ISO8601')

In [3]:
dataframe

,open,high,low,close,volume
date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,7564.90
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.00
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,1117.54
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,51.17
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,599.99
...,...,...,...,...,...
2023-07-04 21:15:00,30794.96,30794.96,30790.20,30790.21,202250.26
2023-07-04 21:16:00,30790.21,30791.08,30790.20,30791.08,210922.27
2023-07-04 21:17:00,30791.08,30791.08,30780.94,30780.95,407579.78


In [4]:
start = "2019-11-01"
end = "2019-11-30"
df = dataframe[start:end]

In [5]:
# load_params = np.load("out.npz", allow_pickle=True)
# load_params.keys()

In [6]:
# load_params = np.load("out-op_control_param.npz", allow_pickle=True)
# control_params = {'tp': [0.6], 'sl': [1.8]}
# for key in load_params:
#     control_params[key] = load_params[key]

# control_params

In [7]:
control_params = {
    'ema_length': np.array([200]),
    'st_factor_main': np.array([16]),
    'st_atr_main': np.array([6]),
    'tf2': np.array([60]),
    'st_factor_tf2': np.array([3]),
    'st_atr_tf2': np.array([10])
}

In [8]:
model = MutantSupertrend()
print(model.params)
# model.update_params(control_params)
# print(model.params)

{'ema_length': array([200]), 'st_factor_main': array([16]), 'st_atr_main': array([6]), 'st_factor_tf2': array([3]), 'st_atr_tf2': array([10]), 'tsv_length': array([13]), 'tsv_ma_length': array([7]), 'adx_length': array([3]), 'adx_threshold': array([60])}


In [9]:
trade_reports = []
sharp_reports = []
drawdown_reports = []
total_sessions = 1
for i in range(total_sessions):
    # backtest_length = 1440
    # start = np.random.choice(len(dataframe) - backtest_length)
    # end = start + backtest_length
    # df = dataframe.iloc[start:end]
    df = dataframe[start:end]
    print(df)
    df = df.groupby(pd.Grouper(freq='5Min')).agg({"open": "first", 
                                                  "high": "max",
                                                  "low": "min",
                                                  "close": "last",
                                                  "volume": "sum"})
    df = df.dropna()
    data = bt.feeds.PandasData(dataname=df, datetime=None,)
    cerebro = bt.Cerebro()
    cerebro.addstrategy(MutantSupertrendBacktrader, model, print_log=True)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='mutant_trade')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='mutant_drawdown')
    cerebro.addanalyzer(
        bt.analyzers.SharpeRatio,
        timeframe=bt.TimeFrame.Days, 
        compression=1, 
        factor=365,
        annualize =True,
        _name='mutant_sharpe'
    )
    cerebro.addsizer(bt.sizers.PercentSizer, percents=99)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0004)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    result = results[0]
    trade_reports.append(result.analyzers.mutant_trade.get_analysis())
    sharp_reports.append(result.analyzers.mutant_sharpe.get_analysis())
    drawdown_reports.append(result.analyzers.mutant_drawdown.get_analysis())
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

                        open     high      low    close     volume
date                                                              
2019-11-01 00:00:00  9140.86  9141.99  9128.84  9130.69  673937.26
2019-11-01 00:01:00  9130.10  9144.70  9129.73  9130.35  361878.05
2019-11-01 00:02:00  9133.62  9133.62  9125.81  9128.08  308489.02
2019-11-01 00:03:00  9126.40  9130.43  9122.59  9125.02  147448.62
2019-11-01 00:04:00  9125.02  9128.52  9116.00  9126.06  393895.36
...                      ...      ...      ...      ...        ...
2019-11-30 23:55:00  7540.02  7547.98  7539.18  7547.53  246636.10
2019-11-30 23:56:00  7547.00  7547.51  7539.21  7545.13  133380.33
2019-11-30 23:57:00  7545.19  7550.01  7545.19  7548.33   88942.93
2019-11-30 23:58:00  7547.89  7548.54  7539.22  7540.61  345227.69
2019-11-30 23:59:00  7540.61  7543.97  7539.52  7541.89  111120.46

[42937 rows x 5 columns]
Starting Portfolio Value: 100000.00
2019-11-01T22:05:00, Open Long, 9270.73
2019-11-01T22:10:00, BUY EX

In [10]:
sharp_reports

[OrderedDict([('sharperatio', 1.9976178305583399)])]

In [11]:
""" Trade report is provided by Backtrader.analyzers.TradeAnalyzer

"""
def get_win_rate(trade_raport):
    """ Get the win rate from trade report
    
    Win rate = number of win trades / number of total trades
    """
    total_trades = trade_raport['total']['total']
    win_trades = trade_raport['won']['total']
    win_rate = win_trades / total_trades
    return win_rate

def get_roi(trade_raport, init_protfolio_value=100000.0):
    """ Get the ROI from trade report

    ROI = PNL / Initial Portfolio Value
    """
    pnl = trade_raport['pnl']['net']['total']
    roi = pnl / init_protfolio_value
    return roi

def get_drawdown(drawdown_report):
    """ Get the drawdown from drawdown report

    """
    drawdown = drawdown_report['drawdown']
    return drawdown

In [12]:
sessions_win = 0
sessions_win_rate = 0
roi_s = []
for i in range(total_sessions):
    trade_report = trade_reports[i]
    if trade_report['total']['total'] > 0:
        drawdown_report = drawdown_reports[i]
        win_rate = get_win_rate(trade_report)
        roi = get_roi(trade_report)
        roi_s.append(roi)
        if roi > 0:
            sessions_win += 1
        drawdown = get_drawdown(drawdown_report)
        print("total_trades: {}, win_rate: {:.2f},  roi: {:.2f}%, drawdown: {:.2f}%.".format(trade_report['total']['total'], win_rate, roi*100, drawdown))
    else:
        print("No trade.")
        
sessions_win_rate = sessions_win/total_sessions * 100
roi_avg = sum(roi_s) / len(roi_s)
print("sessions_win_rate: {:.2f}%, roi_avg: {:.4f}%.".format(sessions_win_rate, roi_avg*100)) 

total_trades: 11, win_rate: 0.36,  roi: 6.69%, drawdown: 7.71%.
sessions_win_rate: 100.00%, roi_avg: 6.6864%.
